In [239]:
## Uncomment the lines below to install the necessary libraries in binder

#import sys
#!{sys.executable} -m pip install pandas
#!{sys.executable} -m pip install matplotlib
#!{sys.executable} -m pip install scipy
#!{sys.executable} -m pip install mip
#!{sys.executable} -m pip install xlrd


import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import re
from pprint import pprint
from scipy import stats
import string
from mip import *

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

### A. Problem parameters
First we import the data from the Excel file into a Pandas dataframe *data*. For convenience, we rename the columns as in the Excel file, using upper-case letters.

In [240]:

data = pd.read_excel('ORscheduling_Complete (1).xlsx',header=None)

new_names = string.ascii_uppercase[:len(data.columns)] ## as many letters as needed
rename_dict = dict(zip(data.columns,new_names))             

data.rename(columns=rename_dict,inplace=True)


Part of the Pandas dataframe *data* is shown below.Please uncomment the last line to see the entire dataframe.

In [241]:
data.head(10)
data.tail(10)
#data

,A,B,C,D,E,F,G,H,I,J,K,L,M,N,O,P,Q,R,S
0,OR Scheduling,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Data,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,# of Surgical Teams per Department per Day,NaN,NaN,NaN,NaN,NaN,NaN,Min # of ORs per day,NaN,NaN,NaN,NaN,NaN,Max # of ORs per day,NaN,NaN,NaN,NaN,NaN
5,NaN,M,T,W,R,F,NaN,M,T,W,R,F,NaN,M,T,W,R,F,NaN
6,Opthalmology,2,2,2,2,2,NaN,0,0,0,0,0,NaN,2,2,2,2,2,NaN
7,Gynecology,3,3,3,3,3,NaN,0,0,0,0,0,NaN,3,3,3,3,3,NaN
8,Oral Surgery,0,1,0,1,0,NaN,0,0,0,0,0,NaN,1,1,1,1,1,NaN
9,Otolaryngology,1,1,1,1,1,NaN,0,0,0,0,0,NaN,1,1,1,1,1,NaN


,A,B,C,D,E,F,G,H,I,J,K,L,M,N,O,P,Q,R,S
31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
32,Constraints,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
33,Daily # ORs assigned is integer,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
34,Total # ORs each day,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
35,UB: Surgical Teams per day,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
36,UB: Daily Dept OR requirement,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
37,LB: Daily Dept OR requirement,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
38,UB: Weekly Dept OR requirement,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
39,LB: Weekly Dept OR requirement,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
40,Departmental Targets,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Next we import the various regions containing the parameters of the problem into Numpy arrays. Note that, contrary to the usual convention in Python, the range or rows defined in each call to *data.loc()*, say *data.loc(n<sub>1</sub>:n<sub>2</sub>,'A')* <u>includes</u> the last row defined in the range, i.e. *n<sub>2</sub>* in this case. 

* The **departments**:



In [242]:
print('Departments:\n')
departments = data.loc[6:10,'A'].values
print(departments)


Departments:

['Opthalmology' 'Gynecology' 'Oral Surgery' 'Otolaryngology'
 'General Surgery']


* The **available surgical teams**, per department, per day</b>:



In [243]:

## Surgical Teams per Department per Day
print('\nTeams availability per department (rows) per day (columns):\n')
available_teams = data.loc[6:10,'B':'F'].values.astype(int)
print(available_teams)



Teams availability per department (rows) per day (columns):

[[2 2 2 2 2]
 [3 3 3 3 3]
 [0 1 0 1 0]
 [1 1 1 1 1]
 [6 6 6 6 6]]


* The **minimum and maximum numbers of operational rooms**, per department, per day</b>:



In [244]:
## Min # of ORs per day
print('\nMinimum number of ORs per department (rows) per day (columns):\n')
min_ors = data.loc[6:10,'H':'L'].values.astype(int)
print(min_ors)

## Max # of ORs per day
print('\nMaximum number of ORs per department (rows) per day (columns):\n')
max_ors = data.loc[6:10,'N':'R'].values.astype(int)
print(max_ors)




Minimum number of ORs per department (rows) per day (columns):

[[0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]]

Maximum number of ORs per department (rows) per day (columns):

[[2 2 2 2 2]
 [3 3 3 3 3]
 [1 1 1 1 1]
 [1 1 1 1 1]
 [6 6 6 6 6]]


* The **minimum and maximum weekly department requirements for operational rooms**, per department:



In [245]:

## Weekly Department Requirements
print('\nWeekly department requirements (1st column: min/ 2nd column: max):\n')
weekly_reqs = data.loc[14:18,'B':'C'].values.astype(int)
print(weekly_reqs)



Weekly department requirements (1st column: min/ 2nd column: max):

[[ 3  6]
 [12 18]
 [ 2  3]
 [ 2  4]
 [18 25]]


* And finally, the **weekly targets per department**, in hours per week:



In [246]:

## Weekly Department Targets
print('\nWeekly targets per department (hours):\n')
weekly_targets = data.loc[14:18,'H'].values.astype(float)
print(weekly_targets)



Weekly targets per department (hours):

[ 39.4 117.4  19.9  26.3 189. ]


### B. Problem formulation

This is an integer programming (IP) problem. The mathematical formulation is as follows. The decision variables are the operational rooms assigned to each department *i*, each day *j*. Let us denote these variables by $x_{ij}, i=1,...,5, j=1,...,5$. 


The **objective function** to **maximize** is the sum of the percentages of the weekly targets achieved. For each department *i*, the total hours worked per week are $8\sum_{j=1}^5 x_{ij}$ and the fraction of the corresponding weekly target for department *i*, $t_i$, is $\frac{8}{t_i}\sum_{j=1}^5 x_{ij}$ 

Therefore, the objective function is: 


$\begin{aligned} max z=\sum_{i=1}^5\frac{8}{t_i}\sum_{j=1}^5 x_{ij} \end{aligned}$


The **constraints** are as follows:

The total number of operating rooms each day has to be less than or equal to 10: 
 
$\begin{aligned} \sum_{i=1}^5 x_{ij}\leq 10 \quad i=1,...,5 \quad(C_1)\end{aligned}$
 
The operating rooms are less than or equal to the available surgical teams, for each department and day: 
 
$\begin{aligned} x_{ij} \leq a_{ij} \quad i=1,...,5 \quad j=1,...,5 \quad(C_2) \end{aligned}$ 

where $a_{ij}$ are the available surgical teams in department *i* and day *j*. 

The operating rooms are also a) greater than or equal to the minimum specified ones, for each department and day, b) less  than or equal to the maximum specified ones, for each department and day:
 
$\begin{aligned} x_{ij} \geq m_{ij}^- \quad i=1,...,5 \quad j=1,...,5 \quad(C_3) \end{aligned}$ 

$\begin{aligned} x_{ij} \leq m_{ij}^+ \quad i=1,...,5 \quad j=1,...,5 \quad(C_4) \end{aligned}$ 

where $m_{ij}^-$ and $m_{ij}^+$ are the minimum and maximum values, per department *i* and day *j*. 



The weekly numbers of operating rooms are between the minimum and maximum requirements: 
 
$\begin{aligned} \sum_{j=1}^5 x_{ij}\geq w_{i}^- \quad i=1,...,5 \quad(C_5)\end{aligned}$ 

$\begin{aligned} \sum_{j=1}^5 x_{ij}\leq w_{i}^+ \quad i=1,...,5 \quad(C_6)\end{aligned}$ 

where $w_{i}^-$ and $w_{i}^+$ are the minimum and maximum, respectively, weekly requirements for operating rooms in department *i*. 

The weekly numbers of operating rooms are less than or equal to the weekly targets: 
 
$\begin{aligned} \sum_{j=1}^5 x_{ij}\leq t_{i} \quad i=1,...,5 \quad(C_7)\end{aligned}$ 

where $t_{i}$ are the weekly targets in department *i*. 

The **decision variables** are:

$x_{ij} $ nonnegative integers: $x_{ij}\in Z^*=\{0\}\cup Z^+ \quad i=1,...,5 \quad  j=1,...,5$. 
 

### C. Modeling the problem using the mip package commands

We use the Python [mip package](https://docs.python-mip.com/en/latest/intro.html). First, we initialize a model *m*, we define the direction of the optimization (*MAXIMIZE*) and select the solver (the default, *CBC*). 

Next, we define the 25 decision variables and give them names $xi\_j$ using the function *add_var()*. In the same command, we select their type as *INTEGER* and also set their lower and upper bounds - from constraints $(C3)$ and $(C4)$ - using the arrays *min_ors* and *max_ors*. We incorporate also constraints $(C2)$ (operating rooms are $\leq $ available surgical teams),
in the upper bounds, using array *available_teams* and use the *minimum* (i.e. tighter) of the two upper bounds.

Note that we use subscripts for the departments (*i*) and days (*j*) from 1 to 5. Therefore, when using the other data of the problem, we deduct one from *i* <br> and/or *j*.

Finally, we define the objective function (property *objective*), according to the formulation in section **B**, using data from array *weekly_targets*. Note that we use the function *var_by_name()* to refer to a variable by its name rather than by an index. Also, we use the function *xsum()* to add several variables together.  


In [247]:

m = Model(name='Operation Rooms',sense=MAXIMIZE, solver_name=CBC) 


## Daily # ORs assigned is integer
x = [m.add_var(name='x'+str(i)+'_'+str(j),var_type=INTEGER, \
               lb = min_ors[i-1,j-1], ub= min(max_ors[i-1,j-1],available_teams[i-1,j-1])) \
     for i in range(1,6) for j in range(1,6)]

m.objective = xsum(xsum(8/weekly_targets[i-1]*m.var_by_name('x'+str(i)+'_'+str(j)) \
                        for j in range(1,6)) for i in range(1,6)) 


Next we add the first constraints $(C1)$ (total number of operating rooms each day $\leq 10$). We use the += shorthand notation for the function *add_constr()* and again use function *xsum()* to add several variables together. 

As with the objective function, we refer to variables using their names, with the function *var_by_name()*. 

In [248]:

## Total # ORs each day
## the total number of operating rooms each day has to be less than or equal to 10.
for j in range(1,6):
    m += xsum(m.var_by_name('x'+str(i)+'_'+str(j)) for i in range(1,6)) <= 10 


The next constraints $(C5)$ and $(C6)$ are very similar and express the constraints due to the minimum / maximum weekly requirements for operating rooms. We use array *weekly_reqs* where the first column (0) has the minimum values and the second column (1) has the maximum ones.

Finally, we add constraints $(C7)$ which put upper bounds (weekly targets) on the weekly numbers of operating rooms, for each department. The values for the right side are taken from array *weekly_targets*.


In [249]:

## UB: Weekly Dept OR requirement
## LB: Weekly Dept OR requirement
for i in range(1,t+1):
    m += xsum(m.var_by_name('x'+str(i)+'_'+str(j)) for j in range(1,d+1)) >= weekly_reqs[i-1,0] #,'week_'+str(j)+' min'
for i in range(1,t+1):
    m += xsum(m.var_by_name('x'+str(i)+'_'+str(j)) for j in range(1,d+1)) <= weekly_reqs[i-1,1] #,'week_'+str(j)+' max'

    
## ----------------------------------------------------------------
## Departmental Targets
for i in range(1,t+1):
    m += 8*xsum(m.var_by_name('x'+str(i)+'_'+str(j)) for j in range(1,t+1)) <= weekly_targets[i-1] #,'weekly_target_'+str(i)




We check the model by writing it in *lp format* in a file *model.lp*. We read back this file and examine it (see next output). We could have written the model also in the more portable *mps format* (by only changing the file extension to *.mps*) but this latter format is more difficult to read. 

We can see that, internally, the problem is turned into a minimization one, by changing the sign of the coefficients of the objective function.

In [250]:

m.write('model.lp')
with open('model.lp', 'r') as f:
    model = f.readlines()
for line in model:
    print(re.sub('\n','',line))
    

\Problem name: Operation Rooms

Minimize
OBJROW: -0.20305 x1_1 -0.20305 x1_2 -0.20305 x1_3 -0.20305 x1_4 -0.20305 x1_5 -0.06814 x2_1 -0.06814 x2_2 -0.06814 x2_3 -0.06814 x2_4 -0.06814 x2_5
 -0.40201 x3_1 -0.40201 x3_2 -0.40201 x3_3 -0.40201 x3_4 -0.40201 x3_5 -0.30418 x4_1 -0.30418 x4_2 -0.30418 x4_3 -0.30418 x4_4 -0.30418 x4_5
 -0.04233 x5_1 -0.04233 x5_2 -0.04233 x5_3 -0.04233 x5_4 -0.04233 x5_5
Subject To
constr(0):  x1_1 + x2_1 + x3_1 + x4_1 + x5_1 <= 10
constr(1):  x1_2 + x2_2 + x3_2 + x4_2 + x5_2 <= 10
constr(2):  x1_3 + x2_3 + x3_3 + x4_3 + x5_3 <= 10
constr(3):  x1_4 + x2_4 + x3_4 + x4_4 + x5_4 <= 10
constr(4):  x1_5 + x2_5 + x3_5 + x4_5 + x5_5 <= 10
constr(5):  x1_1 + x1_2 + x1_3 + x1_4 + x1_5 >= 3
constr(6):  x2_1 + x2_2 + x2_3 + x2_4 + x2_5 >= 12
constr(7):  x3_1 + x3_2 + x3_3 + x3_4 + x3_5 >= 2
constr(8):  x4_1 + x4_2 + x4_3 + x4_4 + x4_5 >= 2
constr(9):  x5_1 + x5_2 + x5_3 + x5_4 + x5_5 >= 18
constr(10):  x1_1 + x1_2 + x1_3 + x1_4 + x1_5 <= 6
constr(11):  x2_1 + x2_2 + x2_

### D. Solving the problem

xxxxx


In [251]:
SearchEmphasis = 1 ; LP_Method = 1 
m.max_gap = 0.05
m.preprocess = 1
status = m.optimize()

In [252]:
if status == OptimizationStatus.OPTIMAL:
    print('optimal solution cost {} found'.format(m.objective_value))
elif status == OptimizationStatus.FEASIBLE:
    print('sol.cost {} found, best possible: {}'.format(m.objective_value, m.objective_bound))
elif status == OptimizationStatus.NO_SOLUTION_FOUND:
    print('no feasible solution found, lower bound is: {}'.format(m.objective_bound))
elif status == OptimizationStatus.INFEASIBLE:
    print('no feasible solution found')
if status == OptimizationStatus.OPTIMAL or status == OptimizationStatus.FEASIBLE:
    print('solution:')
    for v in m.vars:
        #if abs(v.x) > 1e-6: # only printing non-zeros
        print('{} : {}'.format(v.name, v.x))

optimal solution cost 4.456298750836373 found
solution:
x1_1 : 2.0
x1_2 : 0.0
x1_3 : 2.0
x1_4 : 0.0
x1_5 : 0.0
x2_1 : 2.0
x2_2 : 3.0
x2_3 : 3.0
x2_4 : 3.0
x2_5 : 3.0
x3_1 : 0.0
x3_2 : 1.0
x3_3 : 0.0
x3_4 : 1.0
x3_5 : 0.0
x4_1 : 0.0
x4_2 : 0.0
x4_3 : 1.0
x4_4 : 1.0
x4_5 : 1.0
x5_1 : 3.0
x5_2 : 6.0
x5_3 : 4.0
x5_4 : 5.0
x5_5 : 5.0



(to be continued, include also some graphs + some sensitivity analysis)